In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [3]:
from langchain.document_loaders import CSVLoader
loader=CSVLoader(file_path="OutdoorClothingCatalog_1000.csv")

In [4]:
# For creating vector store
from langchain.indexes import VectorstoreIndexCreator
#from langchain.vectorstores import DocArrayInMemorySearch
from langchain_community.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

In [ ]:
index= VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch).from_loaders([loader])

In [6]:
query ="Please list all your shirts with sun protection in a table in markdown and summarize"

In [7]:
response=index.query(query)

In [8]:
display(Markdown(response))



| Name | Description | Sun Protection Rating |
|------|-------------|-----------------------|
| Men's Tropical Plaid Short-Sleeve Shirt | Made of 100% polyester, UPF 50+ rated, front and back cape venting | SPF 50+ |
| Men's Plaid Tropic Shirt, Short-Sleeve | Made of 52% polyester and 48% nylon, UPF 50+ rated, front and back cape venting | SPF 50+ |
| Men's TropicVibe Shirt, Short-Sleeve | Made of 71% nylon and 29% polyester, UPF 50+ rated, front and back cape venting | SPF 50+ |
| Sun Shield Shirt | Made of 78% nylon and 22% Lycra Xtra Life fiber, UPF 50+ rated, wicks moisture | SPF 50+ |

Summary: All four shirts listed have sun protection ratings of SPF 50+ and are made of lightweight, quick-drying fabrics. They also feature front and back cape venting for added breathability.

In [11]:
embeddings= OpenAIEmbeddings()

In [12]:
embed=embeddings.embed_query("Hola! my name art thy Roul")

In [13]:
len(embed)

1536

In [14]:
embed[:5]

[-0.021577656707560115,
 0.0024449621110492667,
 -0.01408614777190077,
 -0.02516514083052186,
 -0.011065802737867392]

In [15]:
loader=CSVLoader(file_path="OutdoorClothingCatalog_1000.csv")
docs=loader.load()

In [16]:
docs[1]

Document(page_content=': 1\nname: Recycled Waterhog Dog Mat, Chevron Weave\ndescription: Protect your floors from spills and splashing with our ultradurable recycled Waterhog dog mat made right here in the USA. \n\nSpecs\nSmall - Dimensions: 18" x 28". \nMedium - Dimensions: 22.5" x 34.5".\n\nWhy We Love It\nMother nature, wet shoes and muddy paws have met their match with our Recycled Waterhog mats. Ruggedly constructed from recycled plastic materials, these ultratough mats help keep dirt and water off your floors and plastic out of landfills, trails and oceans. Now, that\'s a win-win for everyone.\n\nFabric & Care\nVacuum or hose clean.\n\nConstruction\n24 oz. polyester fabric made from 94% recycled materials.\nRubber backing.\n\nAdditional Features\nFeatures an -exclusive design.\nFeatures thick and thin fibers for scraping dirt and absorbing water.\nDries quickly and resists fading, rotting, mildew and shedding.\nUse indoors or out.\nMade in the USA.\n\nHave questions? Reach out to

In [19]:
# Define the file path and chunk size
file_path = "OutdoorClothingCatalog_1000.csv"
chunk_size = 100  # Choose an appropriate chunk size

#Initialize an empty list to store processed documents
all_docs=[]

#Create a CSV Loader
loader=CSVLoader(file_path=file_path)
docs= loader.load_and_split()

db=DocArrayInMemorySearch.from_documents(docs[0:500], embeddings)
db_2=DocArrayInMemorySearch.from_documents(docs[501:1000], embeddings)

In [27]:
len(docs)
#final_db=DocArrayInMemorySearch.from_documents(docs[0:1000], embeddings)

1000

In [60]:
db=DocArrayInMemorySearch.from_documents(docs[0:800], embeddings)
db_2=DocArrayInMemorySearch.from_documents(docs[501:1000], embeddings)

In [30]:
query = "Please suggest a shirt with sunblocking"

In [31]:
combined_results = []
for db in [db,db_2]:
    # Perform search on each individual database
    results = db.similarity_search(query)
    # Combine results, potentially with filtering or merging strategies
    combined_results.extend(results)


In [36]:
len(combined_results)

8

In [37]:
qdocs = "".join([combined_results[i].page_content for i in range(len(combined_results))])

In [54]:
response

{'query': 'Please list all your shirts with sun protection in a table in markdown and summarize each one.',
 'result': "| Shirt Number | Name | Description |\n| --- | --- | --- |\n| 618 | Men's Tropical Plaid Short-Sleeve Shirt | Rated UPF 50+ for superior protection from the sun's UV rays. Made of 100% polyester and is wrinkle-resistant. With front and back cape venting that lets in cool breezes and two front bellows pockets. |\n| 535 | Men's TropicVibe Shirt, Short-Sleeve | Has built-in UPF 50+ and is made of 71% Nylon, 29% Polyester. Wrinkle-resistant with front and back cape venting that lets in cool breezes and two front bellows pockets. |\n| 617 | Girls' Beachside Breeze Shirt, Half-Sleeve | A swim shirt with built-in UPF 50+ and is made of 80% nylon, 20% Lycra Xtra Life. Snag- and fade-resistant fabric holds its shape season after season. |\n| 679 | Women's Tropical Tee, Sleeveless | Has built-in SunSmart™ UPF 50+ and is made of 71% nylon, 29% polyester. With low-profile pockets

In [61]:
from langchain.chains import RetrievalQA

In [64]:
docs = db.similarity_search(query)

In [65]:
docs[0]

Document(page_content=": 374\nname: Men's Plaid Tropic Shirt, Short-Sleeve\ndescription: Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry. Originally designed for fishing, this lightest hot-weather shirt offers UPF 50+ coverage and is great for extended travel. SunSmart technology blocks 98% of the sun's harmful UV rays, while the high-performance fabric is wrinkle-free and quickly evaporates perspiration. Made with 52% polyester and 48% nylon, this shirt is machine washable and dryable. Additional features include front and back cape venting, two front bellows pockets and an imported design. With UPF 50+ coverage, you can limit sun exposure and feel secure with the highest rated sun protection available.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 374})

In [69]:
retriever = db.as_retriever()
llm = ChatOpenAI(temperature = 0.0, model=llm_model)
qdocs = "".join([docs[i].page_content for i in range(len(docs))])
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [70]:
display(Markdown(response))

| Shirt Name | Description |
| --- | --- |
| Men's Plaid Tropic Shirt, Short-Sleeve | This shirt offers UPF 50+ coverage and is great for extended travel. SunSmart technology blocks 98% of the sun's harmful UV rays, while the high-performance fabric is wrinkle-free and quickly evaporates perspiration. Made with 52% polyester and 48% nylon, this shirt is machine washable and dryable. Additional features include front and back cape venting, two front bellows pockets and an imported design. |
| Men's Tropical Plaid Short-Sleeve Shirt | This fabric is made of 100% polyester and is wrinkle-resistant. With front and back cape venting that lets in cool breezes and two front bellows pockets, this shirt is imported and provides the highest rated sun protection possible. |
| Men's TropicVibe Shirt, Short-Sleeve | This Men’s sun-protection shirt with built-in UPF 50+ has the lightweight feel you want and the coverage you need when the air is hot and the UV rays are strong. Made with shell: 71% Nylon, 29% Polyester and lining: 100% Polyester knit mesh. Additional features include wrinkle resistance, front and back cape venting, and two front bellows pockets. |
| Sun Shield Shirt | This high-performance sun shirt is guaranteed to protect from harmful UV rays. Made with 78% nylon and 22% Lycra Xtra Life fiber, this shirt is UPF 50+ rated and wicks moisture for quick-drying comfort. It fits comfortably over your favorite swimsuit and is abrasion-resistant for season after season of wear. | 

All of these shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. They are made with high-performance fabrics that are wrinkle-resistant, quick-drying, and comfortable to wear. Additional features include front and back cape venting, two front bellows pockets, and abrasion resistance.

In [63]:
response = qa_stuff.invoke(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [71]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [72]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [73]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [74]:
display(Markdown(response))

| Shirt Number | Name | Description |
| --- | --- | --- |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | Rated UPF 50+ for superior protection from the sun's UV rays. Made of 100% polyester and is wrinkle-resistant. With front and back cape venting that lets in cool breezes and two front bellows pockets. |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | Rated to UPF 50+ and offers sun protection. Made with 52% polyester and 48% nylon, this shirt is machine washable and dryable. Additional features include front and back cape venting, two front bellows pockets. |
| 535 | Men's TropicVibe Shirt, Short-Sleeve | Built-in UPF 50+ has the lightweight feel you want and the coverage you need when the air is hot and the UV rays are strong. Made with 71% Nylon, 29% Polyester. Additional features include wrinkle resistance, front and back cape venting, and two front bellows pockets. |
| 255 | Sun Shield Shirt | High-performance sun shirt is guaranteed to protect from harmful UV rays. Made with 78% nylon, 22% Lycra Xtra Life fiber. Additional features include moisture-wicking, abrasion resistance, and fits comfortably over your favorite swimsuit. |

The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant. It has front and back cape venting that lets in cool breezes and two front bellows pockets. The Men's Plaid Tropic Shirt, Short-Sleeve is made with 52% polyester and 48% nylon, and has front and back cape venting, and two front bellows pockets. The Men's TropicVibe Shirt, Short-Sleeve has built-in UPF 50+ and is made with 71% Nylon, 29% Polyester. It has wrinkle resistance, front and back cape venting, and two front bellows pockets. The Sun Shield Shirt is made with 78% nylon, 22% Lycra Xtra Life fiber, and is moisture-wicking and abrasion-resistant. It fits comfortably over your favorite swimsuit. All shirts have UPF 50+ sun protection, blocking 98% of the sun's harmful rays.